In [1]:
from fairseq.models.bart import BARTModel
import torch
from rouge import Rouge, FilesRouge
from tqdm import tqdm_notebook as tqdm
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

gpu num:  0


In [2]:
bart = BARTModel.from_pretrained(
    'checkpoints_multi_view',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='../data'
)

2022-04-19 20:30:09 | INFO | fairseq.file_utils | loading archive file checkpoints_multi_view
2022-04-19 20:30:11 | INFO | fairseq.tasks.translation | [source] dictionary: 50264 types
2022-04-19 20:30:11 | INFO | fairseq.tasks.translation | [target] dictionary: 50264 types


RuntimeError: Error(s) in loading state_dict for BARTModel:
	Unexpected key(s) in state_dict: "section_positions.weight", "section_layernorm_embedding.weight", "section_layernorm_embedding.bias", "section.weight_ih_l0", "section.weight_hh_l0", "section.bias_ih_l0", "section.bias_hh_l0", "w_proj_layer_norm.weight", "w_proj_layer_norm.bias", "w_proj.weight", "w_proj.bias", "w_context_vector.weight". 

In [9]:
bart.eval()
#bart.cuda()

BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (dropout_module): FairseqDropout()
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (dropout_module): FairseqDropout()
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout_module): FairseqDropout()
          (activation_dropout_module): FairseqDro

In [10]:
from tqdm import tqdm as tqdm

In [15]:
count = 1
bsz = 8
with open('./fairseq_multi_view/data3/train_sent_trans_cons_label.source') as source, open('./fairseq_multi_view/data3/train_sent_c99_label.source') as source2, open('./train_best_multi_attn_best.hypo', 'wt', encoding='utf-8') as fout:
    s1 = source.readlines()
    s2 = source2.readlines()

    slines = [s1[0].strip()]
    slines2 = [s2[0].strip()]

    for i in tqdm(range(1, len(s1))):
        if count % bsz == 0:
            with torch.no_grad():
                hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
               
            for hypothesis in hypotheses_batch:
                fout.write(hypothesis + '\n')
                fout.flush()
            slines = []
            slines2 = []

        slines.append(s1[i].strip())
        slines2.append(s2[i].strip())

        count += 1

    if slines != []:

        hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
       

        for hypothesis in hypotheses_batch:
            fout.write(hypothesis + '\n')
            fout.flush()

        

  0%|                                                   | 0/450 [00:00<?, ?it/s]/Users/antonio/anaconda3/lib/python3.9/site-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/Users/antonio/anaconda3/lib/python3.9/site-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torc

KeyboardInterrupt: 

In [13]:
from rouge import Rouge, FilesRouge

In [15]:
hyp_path = './test_best_multi_attn_best.hypo'
ref_path = './fairseq_multi_view_code/data3/test_sent_trans_cons_label.target'
hypothesis = []
with open(hyp_path, 'r') as f:
    lines = f.readlines()
    for l in lines:
        hypothesis.append(l[:-1])

reference = []
with open(ref_path, 'r') as f:
    lines = f.readlines()
    for l in lines:
        reference.append(l[:-1])

rouge = Rouge()
print('Test', rouge.get_scores(hypothesis, reference, avg = True))

Test {'rouge-1': {'f': 0.49009429017411965, 'p': 0.5079810307154228, 'r': 0.5198850300191744}, 'rouge-2': {'f': 0.25332958515740567, 'p': 0.26215165975601734, 'r': 0.2721887911613676}, 'rouge-l': {'f': 0.47529740021449296, 'p': 0.49092217200422267, 'r': 0.4978984240638189}}
